In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.applications import mobilenet_v2
from keras.src.applications import MobileNetV2
from tqdm.notebook import tqdm

from models.mobilenetv2.utils import split_functional_model

In [3]:
def normalize_img(img, lbl):
    """Normalizes images: `uint8` -> `float32`."""
    img = tf.image.resize_with_pad(img, 224, 224)
    img = mobilenet_v2.preprocess_input(img)
    return img, lbl

In [4]:
# load imagenet2012 dataset
validation_ds, metadata = tfds.load(
    'imagenet2012',
    split='validation',
    with_info=True,
    as_supervised=True,
)
validation_ds = validation_ds.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
get_label_name = metadata.features['label'].int2str

In [5]:
def representative_dataset():
    # should be 100 to 500 according to documentation
    number_of_samples = 100
    for data in validation_ds.batch(1).take(number_of_samples):
        yield [data[0]]

In [6]:
def quantize_and_save_model(model, name, head_network=None):
    # Convert the model.
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_dataset
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.int8  # or tf.uint8
    converter.inference_output_type = tf.int8  # or tf.uint8
    tflite_model = converter.convert()

    # Save the model.
    with open("models/head/" + str(name) + '.tflite', 'wb') as f:
        f.write(tflite_model)

In [7]:
model = MobileNetV2(weights='imagenet')

In [8]:
print("Save full models")
#quantize_and_save_model(model, "75")
model.save("models/tail/0.keras")

In [9]:
print("Save partial models")
# skip full model with first and last index
for i in tqdm(range(1, 75)):
    head, tail = split_functional_model(model, i)
    #quantize_and_save_model(head, i)
    tail.save("models/tail/" + str(i) + ".keras")